In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.fftpack

In [ ]:
##### from function.py  (given in the in-class assignment)

# set up our time domain
t_start = 0.0
t_stop = 4.0*np.pi

# number of samples - we're going to vary this from ~100 to 10,000
N_samples = 10000

dt = (t_stop-t_start)/N_samples

# time range
t = np.linspace(0,N_samples*dt,N_samples,endpoint=False)

# function without noise
fx = 4.0*np.sin(15.0*2.0*np.pi*t) + 1.5*np.cos(30.0*2.0*np.pi*t) + 3.0*np.cos(60.0*2.0*np.pi*t) 

# function with noise added
fx_noisy = fx + 1.0*np.random.rand(N_samples)

# noisy function with the mean removed (should get rid of f=0 DC offset)
fx_demeaned = fx_noisy - fx_noisy.mean()

plt.plot(t,fx,'r--')
plt.plot(t,fx_noisy,'b.')
#plt.plot(t,fx_demeaned,'kx',alpha=0.5)  # values with mean removed
plt.xlim(0,np.pi/8)

In [ ]:
# get fourier transforms

# noiseless
yf = scipy.fftpack.fft(fx)

# noisy 
yf_noisy = scipy.fftpack.fft(fx_noisy)

# noisy w/mean removed
yf_demeaned = scipy.fftpack.fft(fx_demeaned)

# only plot the first half of the samples - we don't care about the rest, 
# that's all the negative frequency components (since we're using the 
# standard FFT, not the real-only FFT)
xf = np.linspace(0.0,1.0/(2.0*dt), N_samples//2)

# plot everything.  Red line (noisy) shold have some junk in it including a f=0 signal (DC offset).
# Black line (noisy, but mean subtracted) should have no f=0 signal.
plt.plot(xf, 2.0/N_samples * np.abs(yf[0:N_samples//2]),'b-',linewidth=2)
plt.plot(xf, 2.0/N_samples * np.abs(yf_noisy[0:N_samples//2]),'r--')
plt.plot(xf, 2.0/N_samples * np.abs(yf_demeaned[0:N_samples//2]),'k--')
plt.grid()
plt.xlabel(r'Frequency ($\nu$)' )
plt.ylabel(r'F($\nu$)')
plt.title(r"F($\nu$) for various versions of our signal")
plt.xlim(-1,100)
plt.ylim(-.1,.2)

In [ ]:
# now let's try this with the real-only fft (rfft)
# this FFT outputs data somewhat differently; it alternates
# real and imaginary components, so we're going to only one 
# every other cell value, skipping the first one: so, 1, 3, 5, ...
#
# fx, fx_noise are f(time)
# yfr, yfr_noise are F(frequency)
yfr = scipy.fftpack.rfft(fx)
yfr_noisy = scipy.fftpack.rfft(fx_noisy)

xf = np.linspace(0.0, 1.0/(2.0*dt), N_samples//2)


# max frequency (Nyquist frequency, corresponds to 2*delta_t)
xfmax = 1.0/(2.0*dt)

# delta_f per bin (linear in frequency space)
dxf = xfmax/(N_samples/2.0) 

# bounds of our filter (frequency)
f_low = 10.0
f_high = 20.0

# convert to array indices.  note that the factor of 2
# is because we have 2x as many bins as we think because
# the fft keeps both real and imaginary!
N_low = 2*int(f_low/dxf)
N_high = 2*int(f_high/dxf)

# copy our array to keep the original around for reference
# (but you can do this in-place if you want)
yfr_filtered = np.copy(yfr)
yfr_filtered[0:N_low] = 0.0
yfr_filtered[N_high:] = 0.0

yfr_noisy_filtered = np.copy(yfr_noisy)
yfr_noisy_filtered[0:N_low] = 0.0
yfr_noisy_filtered[N_high:] = 0.0


# red is filtered frequency range, blue is unfiltered frequency range
# only plotting real component of real fft: 1, 3, 5, ...
# the 2.0/N_samples is normalizing it (doesn't have to be done, it's just
# good form)
plt.plot(xf,2.0/N_samples * np.abs(yfr[1::2]),'b-')
plt.plot(xf,2.0/N_samples * np.abs(yfr_filtered[1::2]),'r--',linewidth=2)
plt.plot(xf,2.0/N_samples * np.abs(yfr_noisy_filtered[1::2]),'k.',linewidth=2)
plt.xlim(0,100)
plt.ylim(-.1,.2)

In [ ]:
# do inverse fourier transforms to get f(t)_filtered and f(t)_noisy_filtered
# (so frequenchy -> time)
# Since we used the real FFT last time (rfft) we have to make sure to use the
# inverse of rfft() this time, which is irfft()

fxr_filtered = scipy.fftpack.irfft(yfr_filtered)
fxr_noisy_filtered = scipy.fftpack.irfft(yfr_noisy_filtered)

# plot out the filtered function.

t = np.linspace(0,N_samples*dt,N_samples,endpoint=False)

plt.plot(t,fx,'b')
plt.plot(t,fxr_filtered,'r-',linewidth=2)
plt.plot(t,fxr_noisy_filtered,'k--',linewidth=2)
plt.xlim(np.pi/8,np.pi/4)

